In [4]:
import redis
import msgpack
import msgpack_numpy as mnp
pool = redis.ConnectionPool(host='34.69.39.105', port=6379, db=0)
r = redis.Redis(connection_pool=pool)

In [140]:
import numpy as np
from bokeh.models import Circle, ColumnDataSource,Plot,ColorBar
from bokeh.plotting import figure, save
from bokeh.transform import linear_cmap
from bokeh.palettes import YlOrBr
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
import geopandas as gpd

In [141]:
def merc(lon, lat):
    ''' Get mercator projection co-ordinates from latitude and logitude'''
    x=[]
    y=[]
    r_major = 6378137.000
    for i in range(len(lon)):
        x.append(r_major * math.radians(lon[i])) 
        scale = x[i]/lon[i]
        y.append(180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
         lat[i]* (np.pi/180.0)/2.0)) * scale)
    return x, y

In [142]:
# Get data from redis server
predmeans = mnp.unpackb(r.get('predmeans'), raw=True)
predvars = mnp.unpackb(r.get('predvars'), raw=True)
Xtest = mnp.unpackb(r.get('Xtest'), raw=True)
Xtraining = mnp.unpackb(r.get('Xtraining'), raw=True)

In [143]:
# Get latitude and longitude from Xtest and convert to mercator
lon = []
lat = []
for i in range(len(Xtest[0][1])):
    lon.extend(Xtest[0][i].tolist())
    lat.extend(Xtest[1][i].tolist())
x, y = merc(lon,lat)

In [146]:
# Create array of sizes for markers from uncertainty predictions
sizes = 15*np.log(1 + 300/np.sqrt(predvars))
min(sizes)

array([45.60215392])

In [147]:
# Get map tiles
tile_provider = get_provider(CARTODBPOSITRON)

# Create ColumnDataSource from coordinates, sizes and predicted means
source = ColumnDataSource(dict(x=x, y=y, sizes=sizes, means=predmeans))

# Create linear colour map for pollution data
mapper = linear_cmap(field_name='means', palette=YlOrBr[9][::-1] ,low=0 ,high=max(predmeans)[0])

# Plot figure and add tiles 
mercator = figure(title=None, x_range=(min(x), max(x)), y_range=(min(y), max(y)),
           x_axis_type="mercator", y_axis_type="mercator")
mercator.add_tile(tile_provider)

# Create and plot glyphs for predictions
glyph = Circle(x="x", y="y", radius="sizes", line_color=mapper, fill_color=mapper, fill_alpha=.8, line_width=1)
mercator.add_glyph(source, glyph)

# Add colour bar 
color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))
mercator.add_layout(color_bar, 'right')

outfp = r"../newmap.html"

# Save the plot by passing the plot -object and output path
save(obj=mercator, filename=outfp)

'C:\\Projects\\Maps\\newmap.html'